In [2]:
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split

import statsmodels.formula.api as smf
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, matthews_corrcoef

In [3]:
file_path = 'raw_data_phase_1_07_26_2024 (3).csv'  # Replace with your Excel file's path

df = pd.read_csv(file_path)

## SVD

In [4]:
feature_cols1 = [f'y2_{i}' for i in range(1, 41)]

In [5]:
# Prepare the data
feature_cols = [f'y2_{i}' for i in range(1, 81)]
feature_cols1 = [f'y2_{i}' for i in range(1, 41)]
# data = df[feature_cols,'plate']
time_columns = [f'elapsed_time_{i}' for i in range(2, 82)]
columns_to_select = feature_cols + ['plate']+['mutant_ID']+time_columns+['light_regime']

#y_columns = [f'y2_{i}' for i in range(1, 81)]


data = df[columns_to_select]


data2 = data[data[feature_cols].isnull().any(axis=1)]  # Rows with nulls in feature_cols
data1 = data[data[feature_cols].notnull().all(axis=1)]  # Rows with no nulls in feature_cols
data2=data2.dropna(axis=1, how='all')

In [6]:
scaler1 = StandardScaler()
data1_centered = scaler1.fit_transform(data1[feature_cols])

# # Compute the cross-product matrix
# sponge_m = np.dot(sponge_X.T, sponge_X)  # Equivalent to crossprod(sponge.X)

# # Perform SVD
m = np.dot(data1_centered.T, data1_centered)  # Equivalent to crossprod(sponge.X)
U, S, Vt = np.linalg.svd(m, full_matrices=False)

a1 = U[:, 0]  # First left singular vector
b1 = Vt[0, :]  # First right singular vector

# Step 6: Deflate the First Component
t1 = np.dot(data1_centered, a1) / np.linalg.norm(a1)  # Project data onto the first singular vector
c1 = np.dot(data1_centered.T, t1) / np.dot(t1, t1)  # Compute the contribution along the features

# Subtract the first singular component from the data
data1_corrected = data1_centered - np.outer(t1, c1)

# # Step 5: Add back the mean and standard deviation

data1_corrected = scaler1.inverse_transform(data1_corrected)

In [7]:
scaler2 = StandardScaler()
data2_centered = scaler2.fit_transform(data2[feature_cols1])

# # Compute the cross-product matrix
# sponge_m = np.dot(sponge_X.T, sponge_X)  # Equivalent to crossprod(sponge.X)

# # Perform SVD
# U, S, Vt = np.linalg.svd(sponge_m, full_matrices=False)

m = np.dot(data2_centered.T, data2_centered)  # Equivalent to crossprod(sponge.X)
U, S, Vt = np.linalg.svd(m, full_matrices=False)

a2 = U[:, 0]  # First left singular vector
b2 = Vt[0, :]  # First right singular vector



# Step 6: Deflate the First Component
t2 = np.dot(data2_centered, a2) / np.linalg.norm(a2)  # Project data onto the first singular vector
c2 = np.dot(data2_centered.T, t2) / np.dot(t2, t2)  # Compute the contribution along the features

# Subtract the first singular component from the data
data2_corrected = data2_centered - np.outer(t2, c2)

# # Step 5: Add back the mean and standard deviation

data2_corrected = scaler2.inverse_transform(data2_corrected)

In [8]:
data1[feature_cols] = data1_corrected

data2[feature_cols1] = data2_corrected

C:\Users\ruisi\AppData\Local\Temp\ipykernel_8532\636999801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[feature_cols] = data1_corrected


In [9]:
df3 = pd.concat([data1 , data2 ], axis=0, join='outer')
df3

,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,y2_7,y2_8,y2_9,y2_10,...,elapsed_time_73,elapsed_time_74,elapsed_time_75,elapsed_time_76,elapsed_time_77,elapsed_time_78,elapsed_time_79,elapsed_time_80,elapsed_time_81,light_regime
0,0.370014,0.460426,0.231421,0.567164,0.323294,0.457258,0.193308,0.564001,0.342623,0.436677,...,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722,1min-1min
1,0.348125,0.442737,0.234127,0.555001,0.328023,0.404031,0.234283,0.555353,0.358037,0.420842,...,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722,1min-1min
2,0.342679,0.433564,0.246860,0.530723,0.327358,0.434874,0.225201,0.538051,0.358800,0.417534,...,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722,1min-1min
3,0.340775,0.419223,0.219493,0.535295,0.324585,0.406273,0.199348,0.534483,0.319913,0.399981,...,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722,1min-1min
4,0.314920,0.428705,0.242634,0.552833,0.327286,0.422221,0.233100,0.544830,0.328874,0.411528,...,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722,1min-1min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54296,0.408515,0.371996,0.353920,0.360350,0.330177,0.332005,0.304735,0.308433,0.363043,0.343802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20h_ML
54297,0.367060,0.384835,0.367222,0.402322,0.336991,0.302426,0.328691,0.305361,0.355001,0.353719,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20h_ML
54298,0.336303,0.334450,0.330199,0.332625,0.346976,0.350334,0.331781,0.304517,0.325287,0.315586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20h_ML
54299,0.349108,0.350709,0.322636,0.344941,0.334979,0.336065,0.314105,0.308530,0.337792,0.331095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20h_ML


In [10]:
csv_file_path = "SVD_new.csv"  # Specify the desired file path
df3.to_csv(csv_file_path, index=False)

## Combat

In [16]:
# Prepare the data
feature_cols = [f'y2_{i}' for i in range(1, 81)]
# data = df[feature_cols,'plate']
time_columns = [f'elapsed_time_{i}' for i in range(2, 82)]
columns_to_select = feature_cols + ['plate']+['mutant_ID']+['light_regime']+time_columns
data = df[columns_to_select]

# batch = df['plate']
data1 = data[~data.isnull().any(axis=1)]

data2 = data[data.isnull().any(axis=1)]
old=data2['mutant_ID']
data2 = data2.dropna(axis=1)
data2['mutant_ID']=old

In [17]:
from combat.pycombat import pycombat




# Apply ComBat for batch effect correction

corrected_data1 = pycombat(data=data1[feature_cols].T, batch=data1['plate'],mean_only=True)

corrected_data1=corrected_data1.T

feature_cols1 = [f'y2_{i}' for i in range(1, 41)]
corrected_data2 = pycombat(data=data2[feature_cols1].T, batch=data2['plate'], mean_only=True)
corrected_data2=corrected_data2.T

Using mean only version
Found 24 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data
Using mean only version
Found 24 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data


In [18]:
data1[feature_cols] = corrected_data1[feature_cols]

#corrected_data1['mutant_ID'] = data1['mutant_ID']
# feature_cols = [f'elapsed_time_{i}' for i in range(2, 42)]
data2[feature_cols1] = corrected_data2[feature_cols1]

C:\Users\ruisi\AppData\Local\Temp\ipykernel_8532\2365363927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1[feature_cols] = corrected_data1[feature_cols]


In [19]:
df2 = pd.concat([data1 , data2 ], axis=0, join='outer')
df2

,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,y2_7,y2_8,y2_9,y2_10,...,elapsed_time_72,elapsed_time_73,elapsed_time_74,elapsed_time_75,elapsed_time_76,elapsed_time_77,elapsed_time_78,elapsed_time_79,elapsed_time_80,elapsed_time_81
0,0.239540,0.509331,0.222269,0.530151,0.195628,0.505941,0.183710,0.526291,0.214260,0.485023,...,18.070556,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722
1,0.211896,0.495815,0.225012,0.516465,0.194733,0.456871,0.224696,0.516090,0.224024,0.473338,...,18.070556,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722
2,0.195531,0.494561,0.237814,0.489297,0.183395,0.495601,0.215634,0.495842,0.214066,0.477903,...,18.070556,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722
3,0.185595,0.486043,0.210499,0.491742,0.172772,0.472801,0.189796,0.490107,0.167293,0.466140,...,18.070556,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722
4,0.167596,0.489828,0.233589,0.511360,0.183151,0.483075,0.223534,0.502573,0.183967,0.472024,...,18.070556,18.554167,18.571111,19.054722,19.071667,19.555278,19.572222,20.056944,20.073889,20.329722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54300,0.475873,0.471015,0.472363,0.473393,0.450674,0.454677,0.455944,0.442972,0.452062,0.439454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54869,0.286349,0.507711,0.272312,0.507773,0.294532,0.515398,0.270707,0.500480,0.263888,0.497541,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54976,0.207449,0.468209,0.212617,0.439951,0.204388,0.455794,0.229358,0.445905,0.200542,0.462104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55070,0.270634,0.536460,0.239048,0.480322,0.274865,0.496571,0.261954,0.496257,0.257797,0.512334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
csv_file_path = "Combat.csv"  # Specify the desired file path
df2.to_csv(csv_file_path, index=False)

## Removing WT mean

In [24]:
wt_data = df[df['mutant_ID'] == 'WT']

# Select the columns for y2_1 to y2_81
#y_columns = [col for col in wt_data.columns if col.startswith('y2_') and col.endswith(tuple(map(str, range(1, 82))))]
y_columns = [f'y2_{i}' for i in range(1, 81)]
# Group by 'plate' and 'measurement', and compute the mean for WT
wt_mean = wt_data.groupby(['plate', 'light_regime'])[y_columns].mean().reset_index()

# Merge the WT means back with the original data
df_merged = df.merge(wt_mean, on=['plate', 'light_regime'], suffixes=('', '_wt_mean'))

# Normalize the observed Y(II) values by subtracting the WT mean
for col in y_columns:
    df_merged[f'{col}'] = df_merged[col] - df_merged[f'{col}_wt_mean']

# Display the first few rows of the normalized data
#(df_merged[[*[f'{col}_normalized' for col in y_columns]]])

In [28]:
(df_merged[[*[f'{col}' for col in y_columns]]])

,y2_1,y2_2,y2_3,y2_4,y2_5,y2_6,y2_7,y2_8,y2_9,y2_10,...,y2_71,y2_72,y2_73,y2_74,y2_75,y2_76,y2_77,y2_78,y2_79,y2_80
0,0.045231,0.014212,0.011313,0.023050,0.003008,0.023566,-0.021974,0.023366,0.021708,-0.001780,...,0.014581,0.035620,0.023623,0.017068,0.016104,0.009430,0.039412,-0.000098,0.049148,-0.002982
1,0.017587,0.000695,0.014056,0.009363,0.002113,-0.025504,0.019012,0.013165,0.031472,-0.013466,...,0.009573,0.013875,-0.021265,0.001240,0.013405,0.018857,0.020736,0.008004,0.049302,0.009041
2,0.001222,-0.000558,0.026858,-0.017805,-0.009225,0.013226,0.009951,-0.007083,0.021514,-0.008900,...,0.007150,-0.028729,0.006060,0.000282,0.029573,0.004686,0.006941,-0.005779,-0.000920,-0.014536
3,-0.008714,-0.009076,-0.000458,-0.015359,-0.019848,-0.009574,-0.015887,-0.012819,-0.025258,-0.020663,...,-0.003658,-0.008380,-0.014034,-0.018965,0.000881,-0.022946,-0.002098,-0.017323,-0.039683,-0.017699
4,-0.026713,-0.005291,0.022633,0.004258,-0.009469,0.000700,0.017850,-0.000352,-0.008585,-0.014780,...,0.014980,0.015245,-0.001039,-0.009542,0.017742,0.010503,0.002793,0.017812,0.023145,-0.006207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55509,-0.045674,-0.053043,-0.033073,-0.033632,-0.050379,-0.064951,-0.054682,-0.046467,-0.066214,-0.031077,...,-0.037197,-0.039466,-0.041188,-0.060701,-0.044872,-0.033986,-0.043978,-0.060009,-0.054782,-0.044143
55510,-0.063168,-0.084918,-0.067404,-0.075651,-0.059278,-0.040199,-0.039496,-0.078927,-0.059960,-0.043713,...,-0.045493,-0.073075,-0.072917,-0.035351,-0.032708,-0.080853,-0.080033,-0.063993,-0.054771,-0.074208
55511,-0.058797,-0.063889,-0.026481,-0.062798,-0.053514,-0.050621,-0.049888,-0.048728,-0.064771,-0.058706,...,-0.068305,-0.056879,-0.074674,-0.034446,-0.047458,-0.051554,-0.058496,-0.056734,-0.072445,-0.065415
55512,0.084860,0.091282,0.073003,0.076639,0.088894,0.097759,0.079685,0.082771,0.088813,0.084883,...,0.066976,0.086670,0.080712,0.080512,0.107883,0.085047,0.082918,0.075600,0.085630,0.079606


In [29]:
csv_file_path = "WT_mean.csv"  # Specify the desired file path
df_merged.to_csv(csv_file_path, index=False)